## 作業
在鐵達尼資料集中，   
可以用今天範例中提到的三種辨識異常值的方法，  
以 training data 為資料集。  
1.觀察票價是否有異常值的現象?  
2.你覺得找出的異常是真的異常? 你覺得需要做處理嗎?  
(試著講出自己的想法。)  


### 資料
鐵達尼號 train 資料集


In [1]:
# library
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn  as sns
from scipy import stats
import math
import statistics
#讓圖可以在 jupyter notebook顯示
%matplotlib inline

In [2]:
df_train = pd.read_csv("Titanic_train.csv")

### 探索性分析:
*  觀察-是否有異常值  
我們來觀察一下 Fare 變數吧!!

## Q1:觀察票價是否有異常值的現象?

## 方法1
* 進行一個描述性的統計（規定範圍），從而檢視哪些值是不合理的（範圍以外的值）。  


In [17]:
df_train['Fare'].describe()

count    891.000000
mean      32.204208
std       49.693429
min        0.000000
25%        7.910400
50%       14.454200
75%       31.000000
max      512.329200
Name: Fare, dtype: float64

Fare的最小值為0，最大值為512，但是平均為32.2，第一四分位數到第三四分位數都離512很遠，故有異常值

## 方法 2
* 進行3倍標準差原則的計算，從而檢視哪些值是可疑的異常值。  


In [18]:
def outliers_z_score(ys,times):
    mean_y = np.mean(ys)
    stdev_y = np.std(ys)
    z_scores = [(y - mean_y) / stdev_y for y in ys]
    return np.where(np.abs(z_scores) > times)

out_index=outliers_z_score(df_train['Fare'],3)
print(out_index)
print("用第二種方法的找出的 outlier 有哪些?(3倍標準差)")
print(df_train.loc[out_index[0],'Age'])

(array([ 27,  88, 118, 258, 299, 311, 341, 377, 380, 438, 527, 557, 679,
       689, 700, 716, 730, 737, 742, 779]),)
用第二種方法的找出的 outlier 有哪些?(3倍標準差)
27     19.0
88     23.0
118    24.0
258    35.0
299    50.0
311    18.0
341    24.0
377    27.0
380    42.0
438    64.0
527     NaN
557     NaN
679    36.0
689    15.0
700    18.0
716    38.0
730    29.0
737    35.0
742    21.0
779    43.0
Name: Age, dtype: float64


## 方法3-盒鬚圖判別法(IQR method)

In [19]:
def outliers_iqr(ys,times):
    #注意 使用 np.percentile 要去除 nan 要不然計算出來會錯誤，所以我們採用下者的程式 np.nanpercentile
    #quartile_1, quartile_3 = np.percentile(ys, [25, 75])
    quartile_1, quartile_3 = np.nanpercentile(ys, [25, 75])
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * times)
    upper_bound = quartile_3 + (iqr * times)
    return np.where((ys > upper_bound) | (ys < lower_bound))

out_index2=outliers_iqr(df_train['Age'],1.5)
print(out_index2)
print("用第三種方法的找出的 outlier 有哪些?(1.5 倍IQR)")
print(df_train.loc[out_index2[0],'Age'])

(array([ 33,  54,  96, 116, 280, 456, 493, 630, 672, 745, 851]),)
用第三種方法的找出的 outlier 有哪些?(1.5 倍IQR)
33     66.0
54     65.0
96     71.0
116    70.5
280    65.0
456    65.0
493    71.0
630    80.0
672    70.0
745    70.0
851    74.0
Name: Age, dtype: float64


## Q2:你覺得找出的異常是真的異常? 你覺得需要做處理嗎?


不一定，要依照要分析的目標去思考，這些值是否會影響結果，是否需要做處理。
每種分析找異常值的方法也不一定，所以需要去做改變才能找出真的需要處理的異常值。